This notebook aims to reproduce results of the paper: 

**Detection of Adversarial Attacks by Analyzing Deep Features with Multivariate Data Algorithms**




It allows to execute XGBoost on the hidden layers of MNIST-2 and MNIST-1, and reproduce our results.


For each individual layer and each individual attack:

- it trains XGBoost
- it attempts detection also on the other attacks
 


It requires the layers available at https://drive.google.com/drive/folders/1JsV45ooRlk5CpqFCPy-uR4iBB3Nbqx08?usp=sharing ,
folder MNIST-2 or MNIST-1


We assume the MNIST-1 or MNIST-2 folder is located at path BASE (e.g., BASE=/home/whatever/MNIST-2 )


We recommend to use the conda environment available on the github.

In [3]:
#configuration parameters, check carefully

TEST_SIZE_SPLIT= 0.2 #we recommend 0.2
DATASET="mnist2" #set either to "mnist1" or "mnist2"
LOGFILE="mnist_2.csv" #logfile, choose whatever name you like
RESULTS='/home/whatever/' #path to logfile
BASE='/home/whatever/MNIST-2/' #must point to MNIST-1 or MNIST-2 folders

In [1]:
#from here on, everything should just run smoothly -- if you find errors, please contact us!
ATTACK=['bim_03', 'deep', 'fgsm_03', 'carlini_l2']
SUFFIX='.npy'
METHOD="XGBOOST"

In [2]:
from sklearn.metrics import accuracy_score

import numpy as np
import sys
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
import csv
from matplotlib import pyplot as plt
import math
from pathlib import Path
from ipywidgets import IntProgress
from IPython.display import display
import os
import pandas as pd
import time

Just code to load the .npy files stored

In [9]:
def list_files(name):
    BASE_DIR=BASE+name
    file_list=[]
    for fx in os.listdir(BASE_DIR):
        f = os.path.join(BASE_DIR, fx)
        if os.path.isfile(f):
            file_list.append(f)
    return file_list

In [10]:
#get list of layers by number
train_1=list_files("/train/")
number_list=[]
for i in train_1:
    file_name=os.path.basename(i)
    number = file_name.partition("_")[0]
    number= int(number)
    if number not in number_list:
        number_list.append(number)
print(number_list)

FileNotFoundError: [Errno 2] No such file or directory: '/home/whatever/MNIST-2//train/'

In [12]:
#load train data for a specific layer
def load_train(L, DATASET):
    if(DATASET=="mnist2"):
        p = Path(BASE+"/train/layer_"+str(L)+".npy")
    else:
        p = Path(BASE+"/train/layer_"+str(L)+".npy")
    with p.open('rb') as f:
        nptmp=np.load(f)
    return nptmp

def load_test(L, DATASET):
    if(DATASET=="mnist2"):
        p = Path(BASE+"/test/layer_"+str(L)+".npy")
    else:
        p = Path(BASE+"/test/mnist_test_layers/layer_"+str(L)+"_test_data.npy")
    with p.open('rb') as f:
        nptmp=np.load(f)
    return nptmp

#prefix, layer
def load_attack(attack, L):
    if(DATASET=="mnist2"):
        p = Path(BASE+"/"+str(attack)+"/layer_"+str(L)+".npy")
    else:
        p = Path(BASE+"/"+str(attack)+"/"+str(L)+"_layers.npy")
    with p.open('rb') as f:
        nptmp=np.load(f)
    return nptmp

Transform the hidden features of a layer in a vector, i.e., in a row. This way we can apply algorithms for tabular data (XGBoost in this case)

In [5]:
def prod(val): 
    res = 1 
    for ele in val: 
        res *= ele 
    return res  

In [14]:
#def get_max_min(array):
#    minimum=np.min(array)
#    maximum=np.max(array)
#    return minimum, maximum

#def normalize(array, minimum, maximum):
#    if(NORMALIZE_DATA==True):
#        return (array - minimum) / (maximum - minimum)#(array - np.min(array)) / (np.max(array) - np.min(array)) 
#    else:
#        return array

def linearize(numpy_linearized):
    shape_tuple=numpy_linearized.shape[1:]
    row_length=prod(list(shape_tuple))
    numpy_linearized=numpy_linearized.reshape(
        numpy_linearized.shape[0], row_length)
    return numpy_linearized

some functions to prepare for logging

In [15]:
def write_log(logfile, METHOD, layer, attack, attack1, DATASET, shape, accuracy, tn, fp, fn, tp):
    logfile.write(METHOD+", "+
            layer +", "+
            attack +", "+
            attack1 +", "+
            str(DATASET)+",  "+
            str(shape)+",  "+
            str(accuracy)+",  "+
            str(tn)+",  "+
            str(fp)+",  "+
            str(fn)+",  "+
            str(tp)+",  "+"\n")
    logfile.flush()

def write_sentence(logfile, sentence):
    logfile.write(sentence)
    logfile.write("\n")
    logfile.flush()

In [45]:
f = open(RESULTS+LOGFILE, "a")

In [46]:
f.write("ALG, LAYER, ATTACK, TRAINED ON, LAYER SHAPE, ACCURACY, TN, FP, FN, TP  \n")
f.flush()

These creates the train and test datasets.

For example, they match each normal data point to 0, and each attack data point to 1.

In [44]:
def create_normal(normal_x):
    normal_y=np.empty([normal_x.shape[0], 1])
    normal_y.fill(0)
    return normal_x, normal_y

def create_attack(attack_x):
    attack_y=np.empty([attack_x.shape[0], 1])
    attack_y.fill(1)
    return attack_x, attack_y

def create_test_set(numpy_test_x,attack_test_x, attack_test_y):
    numpy_test_x=linearize(numpy_test_x)
    numpy_test_y=np.empty([numpy_test_x.shape[0], 1])
    numpy_test_y.fill(0)
    merged_x=np.concatenate((numpy_test_x, attack_test_x), axis=0)
    merged_y=np.concatenate((numpy_test_y, attack_test_y), axis=0)
    df=pd.DataFrame(merged_x)
    return df , merged_y

Now the training and evaluation can start:

- take a layer
- take an attack
- organize a train/test set
- do supervised train using XGBOOST
- do predict on the test set (legitimate images + attack images)
- do predict on all the other attacks

In [48]:
for layer in number_list: #for each layer
    x_train=load_train(layer)
    x_train=linearize(x_train)
#    np.random.shuffle(x_train) --> no reason to shuffle
    x_train_normal, y_train_normal=create_normal(x_train)
    x_test=load_test(layer)
    x_test=linearize(x_test)
#    np.random.shuffle(x_train) --> no reason to shuffle
    x_test_normal, y_test_normal=create_normal(x_test)
    print(x_train.shape)
    if(x_train.shape[1]>32800):
        print("skipping, too large size; consider using avgpooling on the layer")
        continue

    for attack in ATTACK: #for each attack
        xgbC=xgb.XGBClassifier(nthread=8)
        try:
            attack_data=load_attack(attack, layer)
        except:
            print("could not load {} for layer {}".format(attack, layer)) #should never see this
            continue
        attack_data=linearize(attack_data)
#        np.random.shuffle(attack_data) --> no reason to shuffle
        x_attack, y_attack=create_attack(attack_data)
        x_train_attack, y_train_attack=create_attack(x_attack[0: round(x_attack.shape[0]*(1-TEST_SIZE_SPLIT))]) #divide the attack set in two parts, one is used for training, the other for test

        #create training set
        x_train=np.concatenate((x_train_normal, x_train_attack), axis=0)
        y_train=np.concatenate((y_train_normal, y_train_attack), axis=0)
        #create test set
        x_test_attack, y_test_attack=create_attack(x_attack[round(x_attack.shape[0]*(1-TEST_SIZE_SPLIT)):])
        x_test=np.concatenate((x_test_normal, x_test_attack), axis=0)
        y_test=np.concatenate((y_test_normal, y_test_attack), axis=0)

        #fit
        xgbC.fit(x_train,y_train)

        write_sentence(f, "prediction on test set (predict is done on the test set + the attack used for training, no unknowns)")
        final_preds=xgbC.predict(x_test)
        accuracy=accuracy_score(y_test, final_preds)
        tn, fp, fn, tp = confusion_matrix(y_test, final_preds, labels=[0,1]).ravel()
        
        write_log(f, METHOD, "layer_"+str(layer), attack, "train set + {}".format(attack),
                  DATASET, x_test.shape[1:],  accuracy, tn, fp, fn, tp)
        write_sentence(f, "now we perform prediction on all the adversarial images (attack by attack)")

        for attack1 in ATTACK:
            try:
                attack_data=load_attack(attack1, layer)
            except:
                continue
            x_attack=linearize(attack_data)
#            np.random.shuffle(x_attack)
            x_attack, y_attack=create_attack(x_attack)
            final_preds=xgbC.predict(x_attack)
            accuracy=accuracy_score(y_attack, final_preds)
            tn, fp, fn, tp = confusion_matrix(y_attack, final_preds, labels=[0,1]).ravel()
            write_log(f, METHOD, "layer_"+str(layer), attack1, "train set + {}".format(attack),
                      DATASET, x_attack.shape[1:], accuracy, tn, fp, fn, tp)


(55000, 128)
/home/notebook/neuron/aggregated/bim_03/layer_9.npy


/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for

[13:55:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/home/notebook/neuron/aggregated/bim_03/layer_9.npy
/home/notebook/neuron/aggregated/carlini_l2/layer_9.npy
/home/notebook/neuron/aggregated/deep/layer_9.npy
/home/notebook/neuron/aggregated/fgsm_03/layer_9.npy
/home/notebook/neuron/aggregated/carlini_l2/layer_9.npy
could not load carlini_l2 for layer 9
/home/notebook/neuron/aggregated/deep/layer_9.npy


/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for

[13:56:03] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/home/notebook/neuron/aggregated/bim_03/layer_9.npy
/home/notebook/neuron/aggregated/carlini_l2/layer_9.npy
/home/notebook/neuron/aggregated/deep/layer_9.npy
/home/notebook/neuron/aggregated/fgsm_03/layer_9.npy
/home/notebook/neuron/aggregated/fgsm_03/layer_9.npy


...

[14:19:25] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/home/notebook/neuron/aggregated/bim_03/layer_2.npy
/home/notebook/neuron/aggregated/carlini_l2/layer_2.npy
/home/notebook/neuron/aggregated/deep/layer_2.npy
/home/notebook/neuron/aggregated/fgsm_03/layer_2.npy
/home/notebook/neuron/aggregated/carlini_l2/layer_2.npy
could not load carlini_l2 for layer 2
/home/notebook/neuron/aggregated/deep/layer_2.npy


/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for

[14:19:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/home/notebook/neuron/aggregated/bim_03/layer_2.npy
/home/notebook/neuron/aggregated/carlini_l2/layer_2.npy
/home/notebook/neuron/aggregated/deep/layer_2.npy
/home/notebook/neuron/aggregated/fgsm_03/layer_2.npy
/home/notebook/neuron/aggregated/fgsm_03/layer_2.npy


/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/notebook/anaconda3/envs/fastai/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for

[14:19:41] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/home/notebook/neuron/aggregated/bim_03/layer_2.npy
/home/notebook/neuron/aggregated/carlini_l2/layer_2.npy
/home/notebook/neuron/aggregated/deep/layer_2.npy
/home/notebook/neuron/aggregated/fgsm_03/layer_2.npy
